In [1]:
import numpy as np
import pandas as pd
import speech_recognition as sr
from gtts import gTTS
import os
import playsound #pip install playsound==1.2.2
import datetime
import transformers
import json
from __future__ import print_function
import random
from transformers import logging as hf_logging


In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

In [3]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\henis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\henis\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\henis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\henis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\henis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# To disable the "Some layers from the model checkpoint at bert-base-cased were not used when initializing"
hf_logging.set_verbosity_error() 

In [5]:
# Build an AI
class ChatBot():
    @classmethod
    def __init__(self,name):
        print("--- Starting up",name,"---")
        self.name = name
        
    @classmethod   
    def speech_to_text(self):
            recognizer = sr.Recognizer()
            with sr.Microphone() as source:
                 recognizer.adjust_for_ambient_noise(source, duration=1)            
                 print("Please say something...")
                 audio = recognizer.listen(source)
            try:
                 self.text = recognizer.recognize_google(audio)
                 print("me --> ", self.text)
            except:
                 print("me -->  ERROR")   
                    
    @classmethod
    def wake_up(self,text):
        self.text = text
        return True if self.name.lower() in self.text.lower() else False
   
        os.system("start res.mp3")
        os.remove("res.mp3")
        
    @staticmethod
    def text_to_speech(text):
        print("ai --> ", text)
        # speaker = gTTS(text=text, lang="en", slow=False)
        speaker = gTTS(text=text, lang="en")
        filename = "res.mp3"
        speaker.save(filename)
        playsound.playsound(filename)
        os.remove(filename)
        
    @staticmethod
    def action_time():
        return datetime.datetime.now().time().strftime('%H:%M %p')


In [6]:
if __name__ == "__main__":
    ai = ChatBot(name="Joshua")
    
    while True:
        ai.speech_to_text()

        # Normalize casings for the user text which had been converted from user voice
        texts = [ai.text]
        case_norm = [sentence.lower() for sentence in texts]
        
        # Tokenize the reviews using NLTKs word_tokenize function.
        tokens = []
        for item in case_norm:
            tokens.append(word_tokenize(item)) 
            
        # Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.
        token_tag = []
        for token in tokens:
            token_tag.append(pos_tag(token))
            
            
        token_noun = []
        for token in token_tag:
            for tok,tag in token:
                if tag.startswith('NN'):
                    token_noun.append((tok,tag)) 
        
        #Lemmatize.
        #Different forms of the terms need to be treated as one.
        #No need to provide POS tag to lemmatizer for now.
        lemmatizer = WordNetLemmatizer()
        token_lemmatized = [lemmatizer.lemmatize(tokens[0]) for tokens in token_noun]
        
        # Remove stopwords and punctuation (if there are any).
        stop_words = stopwords.words('english')        
        token_stop = []
        token_stop.append([token for token in token_lemmatized if not token in stop_words])
        
        
        puctuation_list = list(punctuation)
        token_punct = []
        for token in token_stop:
            if not token in puctuation_list:
                token_punct.append(token)
        token_punct = np.array(token_punct).ravel()      
        
        # Opening the file which has the commands to be included into our chatBot 
        f = open('commands.json')
        jsn = json.load(f)
        jsn_array = jsn['intents'] 
        
        # Creating the JSON array into dataframe
        # separating the tag, patterns and responses into separate arrays and
        # mapping the corresponding three items into a list
        
        df = pd.DataFrame(jsn_array)
        pat = [pattern for pattern in df.patterns]
        tag = [tag for tag in df.tag]
        res = [response for response in df.responses]
        tuples = list(map(lambda x,y,z: (x,y,z), tag, pat,res))
        
        # Choosing the correct tag,pattern and responses according to the user text which had been converted from the user voice
        p = []
        t = []
        r = []
        for tag,patterns,response in tuples:
            for st in patterns:
                if any(x in st for x in token_punct):
                    r.append(response)
                    t.append(tag)
                    p.append(st)
                    respond = np.array(r).ravel()
                    
            
        # Calling the wake_up definition with the user voice or text and getting the name as response
        temp = ai.wake_up(ai.text)
 
        #If the name
        if temp is True:
            res = "Hello I am Joshua the AI, what can I do for you?"
            
        # If the user voice asks for "time"
        elif "time" in ai.text:
            res = ai.action_time()
            
       # If user asks for any of the classes from the file the chatbot responds with one of the responses from the JSON file
        elif len(p)!=0: 
            res = np.random.choice(respond)            

        else:
            res = "error in calling"
            
        # Calling the definition which converts the AI text into speech
        ai.text_to_speech(res)



--- Starting up Joshua ---
Please say something...
me -->  hello Joshua
ai -->  Hello I am Joshua the AI, what can I do for you?
Please say something...
me -->  what support is offered
ai -->  Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies
Please say something...
me -->  drug causing adverse
ai -->  Navigating to Adverse drug reaction module
Please say something...
me -->  find me a pharmacy
ai -->  Please provide pharmacy name
Please say something...
me -->  thank you for helping
ai -->  Happy to help!


KeyboardInterrupt: 